In [14]:
from pathos.multiprocessing import Pool
import os
import requests
import json
import pandas as pd
from datetime import datetime

In [13]:
# Fetch real-time data function
def fetch_realtime_data(route_id, api_key):
    """Fetch real-time data for a specific bus route."""
    print(f"Fetching data for route: {route_id}")
    
    endpoint = "https://bustime.mta.info/api/siri/vehicle-monitoring.json"
    params = {"key": api_key, "LineRef": route_id}

    # Print the full URL for debugging
    full_url = requests.Request('GET', endpoint, params=params).prepare().url
    print(f"Requesting URL: {full_url}")
    
    try:
        response = requests.get(endpoint, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        vehicles = data.get("Siri", {}).get("ServiceDelivery", {}).get("VehicleMonitoringDelivery", [{}])[0].get("VehicleActivity", [])
        
        if not vehicles:
            print(f"No vehicles found for route: {route_id}")
            return None

        records = []
        for vehicle in vehicles:
            journey = vehicle["MonitoredVehicleJourney"]
            location = journey["VehicleLocation"]
            records.append({
                "Route ID": route_id,
                "Vehicle ID": journey["VehicleRef"],
                "Latitude": location.get("Latitude"),
                "Longitude": location.get("Longitude"),
                "Timestamp": datetime.now().isoformat()
            })

        return pd.DataFrame(records)
    except Exception as e:
        print(f"Error fetching data for route {route_id}: {e}")
        return None

# Collect data for multiple routes function
def collect_data_for_routes(route_ids, api_key):
    """Collect real-time data for multiple routes using pathos."""
    print("Starting data collection with Pathos...")
    
    def fetch_with_key(route_id):
        return fetch_realtime_data(route_id, api_key)
    
    with Pool(processes=4) as pool:
        results = pool.map(fetch_with_key, route_ids)

    # Combine results
    all_data = [df for df in results if df is not None]
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        print("No data collected for any routes.")
        return pd.DataFrame()

def save_data_to_csv(data, output_file="realtime_data.csv"):
    """Save data to CSV."""
    if not data.empty:
        if os.path.exists(output_file):
            data.to_csv(output_file, mode='a', header=False, index=False)
        else:
            data.to_csv(output_file, index=False)
        print(f"Data saved to {output_file}.")
    else:
        print("No data to save.")

# if __name__ == "__main__":
#     # Fetch API Key
#     api_key = os.getenv("MTA_API_KEY")
#     if not api_key:
#         raise ValueError("API Key not found. Set MTA_API_KEY as an environment variable.")

#     # List of routes to collect data for
#     route_ids = [
#         "MTA%20NYCT_Q1", "MTA%20NYCT_Q2", "MTA%20NYCT_Q3", "MTA%20NYCT_Q4",
#         "MTA%20NYCT_Q5", "MTA%20NYCT_Q12", "MTA%20NYCT_Q13", "MTA%20NYCT_Q20A",
#         "MTA%20NYCT_Q20B", "MTA%20NYCT_Q27", "MTA%20NYCT_Q28", "MTA%20NYCT_Q30",
#         "MTA%20NYCT_Q31", "MTA%20NYCT_Q36", "MTA%20NYCT_Q43", "MTA%20NYCT_Q44+",
#         "MTA%20NYCT_Q46", "MTA%20NYCT_Q76", "MTA%20NYCT_Q77", "MTA%20NYCT_Q83",
#         "MTA%20NYCT_Q84", "MTA%20NYCT_Q85", "MTA%20NYCT_Q88"
#     ]

#     # Collect and save data
#     data = collect_data_for_routes(route_ids, api_key)
#     save_data_to_csv(data)

if __name__ == "__main__":
    api_key = os.getenv("MTA_API_KEY")
    print(fetch_realtime_data("MTA%20NYCT_Q27", api_key))

Fetching data for route: MTA%20NYCT_Q27
Requesting URL: https://bustime.mta.info/api/siri/vehicle-monitoring.json?key=9ab1f40b-0b53-4169-8ca1-ce89d8094e9a&LineRef=MTA%2520NYCT_Q27
No vehicles found for route: MTA%20NYCT_Q27
None


In [18]:
api_key = "MTA_API_KEY"
route_id = "MTA NYCT_Q43"  # Replace with a valid route ID

endpoint = "https://bustime.mta.info/api/siri/vehicle-monitoring.json"
params = {"key": api_key, "LineRef": route_id}

response = requests.get(endpoint, params=params)

print(response.url)
print(f"Status Code: {response.status_code}")
print("Response Content:")
print(json.dumps(response.json(), indent=4))

https://bustime.mta.info/api/siri/vehicle-monitoring.json?key=MTA_API_KEY&LineRef=MTA+NYCT_Q43
Status Code: 403
Response Content:
{
    "Siri": {
        "ServiceDelivery": {
            "ResponseTimestamp": "2025-01-07T22:31:59.402-05:00",
            "VehicleMonitoringDelivery": [
                {
                    "ResponseTimestamp": "2025-01-07T22:31:59.402-05:00",
                    "ErrorCondition": {
                        "OtherError": {
                            "ErrorText": "API key is not authorized."
                        },
                        "Description": "API key is not authorized."
                    }
                }
            ]
        }
    }
}
